In [19]:
import numpy as np
import pandas as pd
from fastai2.collab import *
from fastai2.tabular.all import *

In [20]:
ratings = pd.read_csv('train.csv')
ratings.overall = ratings.overall.astype('int32')

In [21]:
ratings = ratings[['userid','itemid','overall','unixReviewTime']]
ratings.columns = ['user','item','rating','timestamp']

In [22]:
ratings.head()

,user,item,rating,timestamp
0,102179,37138,5,1475539200
1,3625,17322,5,1456790400
2,39495,5600,5,1466899200
3,72854,30249,5,1378339200
4,74859,4349,5,1473811200


In [23]:
dls = CollabDataLoaders.from_df(ratings, item_name='item', bs=64)
dls.show_batch()

,user,item,rating
0,16963,13420,2
1,14538,24686,5
2,88256,6113,5
3,125235,27000,3
4,88295,37933,5
5,66963,29757,5
6,44211,7558,5
7,100406,27500,5
8,21559,6271,5
9,85962,5767,5


In [24]:
dls.classes

{'user': (#126967) ['#na#',0,1,2,3,4,5,6,7,8...],
 'item': (#41223) ['#na#',0,1,2,3,4,5,6,7,8...]}

In [25]:
n_users = len(dls.classes['user'])
n_items = len(dls.classes['item'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
item_factors = torch.randn(n_items, n_factors)

In [26]:
learn = collab_learner(dls, n_factors=50, y_range=(0,5.5))

In [27]:
learn.fit_one_cycle(5, 1e-3, wd=0.1)

In [15]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(126949, 50)
  (i_weight): Embedding(41217, 50)
  (u_bias): Embedding(126949, 1)
  (i_bias): Embedding(41217, 1)
)

находим самые высокооцененные

In [16]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['movie'][i] for i in idxs]

[5195, 24556, 22208, 23540, 1919]

находим ближайшего похожего

In [31]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['movie'].o2i['22208']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[:1]
dls.classes['movie'][idx]

(#1) ['#na#']